In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms

In [ ]:
!pip install qutip
!pip install qinfer

     |████████████████████████████████| 14.6MB 195kB/s 
     |████████████████████████████████| 133kB 14.6MB/s 


In [ ]:
from qutip import*
import numpy as np
#import qinfer
import matplotlib.pyplot as plt

In [ ]:
#from qinfer.tomography import pauli_basis,gell_mann_basis
#from qinfer.tomography import GinibreDistribution
import math
#from qutip import*
import numpy as np
import matplotlib.pyplot as plt


In [ ]:
'''
H= Qobj([[1],[0]])
V= Qobj([[0],[1]])
D= (H+V)/pow(2,0.5)
A= (H-V)/pow(2,0.5)
R= (H+V*1j)/pow(2,0.5)
L= (H-V*1j)/pow(2,0.5)
H_rho= H * H.dag()
V_rho= V * V.dag()
D_rho= D * D.dag()
A_rho= A * A.dag()
R_rho= R * R.dag()
L_rho= L * L.dag()
'''

H= np.array([[1],[0]])
V= np.array([[0],[1]])
D= (H+V)/pow(2,0.5)
A= (H-V)/pow(2,0.5)
R= (H+V*1j)/pow(2,0.5)
L= (H-V*1j)/pow(2,0.5)
H_rho= H *np.transpose(np.conj(H))
V_rho= V * np.transpose(np.conj(V))
D_rho= D * np.transpose(np.conj(D))
A_rho= A * np.transpose(np.conj(A))
R_rho= R * np.transpose(np.conj(R))
L_rho= L * np.transpose(np.conj(L))


#Projector matrix
P =         np.array([[np.kron(H_rho,H_rho),np.kron(H_rho,V_rho),np.kron(V_rho,V_rho),np.kron(V_rho,H_rho),np.kron(V_rho,R_rho),np.kron(V_rho,L_rho)],
              [np.kron(H_rho,L_rho),np.kron(H_rho,R_rho),np.kron(H_rho,D_rho),np.kron(H_rho,A_rho),np.kron(V_rho,A_rho),np.kron(V_rho,D_rho)],
              [np.kron(A_rho,D_rho),np.kron(A_rho,A_rho),np.kron(D_rho,A_rho),np.kron(D_rho,D_rho),np.kron(D_rho,R_rho),np.kron(D_rho,L_rho)],
              [np.kron(A_rho,L_rho),np.kron(A_rho,R_rho),np.kron(A_rho,H_rho),np.kron(A_rho,V_rho),np.kron(D_rho,V_rho),np.kron(D_rho,H_rho)],
              [np.kron(R_rho,H_rho),np.kron(R_rho,V_rho),np.kron(L_rho,V_rho),np.kron(L_rho,H_rho),np.kron(L_rho,R_rho),np.kron(L_rho,L_rho)],
              [np.kron(R_rho,L_rho),np.kron(R_rho,R_rho),np.kron(R_rho,D_rho),np.kron(R_rho,A_rho),np.kron(L_rho,A_rho),np.kron(L_rho,D_rho)]])



In [ ]:
def matrix_minor(arr, i, j):
    return np.delete(np.delete(arr,i,axis=0), j, axis=1)

In [ ]:
#defining a random qubit

n_rhos = 200
dataset= 200 * n_rhos

#labels_Tau = np.zeros((200*100,1,4,4),dtype = 'complex_')
labels_Tau = np.empty((dataset,16))
input_M_noisy = np.empty((dataset,1,6,6),dtype = 'complex_')
#print(labels_Tau.shape)


for k in range(n_rhos):
  #rand_ket=rand_ket_haar(4)
  #rho = rand_ket*rand_ket.dag()
  G= (np.random.normal(0,1,(4,4))) + 1j*(np.random.normal(0,1,(4,4)))
  rho = (np.matmul(G,np.conjugate(np.transpose(G))))/np.trace(np.matmul(G,np.conjugate(np.transpose(G))))
  #rho= rand_dm_ginibre(4)
  #print(rho)
  M = np.zeros((6,6))
  
  #print((rho * P[1][0].full()).trace())
  for i in range(6):
    for j in range(6):
      M[i][j] = np.trace((np.matmul(rho,P[i][j])))
  
  #Noise unitary rotation matrix
  mu=0
  sigma = math.pi/6
  n_noisy = 200
  for ni in range(n_noisy):
    phi=np.random.normal(mu, sigma)
    theta=np.random.normal(mu, sigma)
    zeta=np.random.normal(mu, sigma)

    #U=Qobj([[np.exp(1j*phi/2)*np.cos(theta) ,-1j* np.exp(1j*zeta)*np.sin(theta)],[-1j* np.exp(-1j*zeta)*np.sin(theta),np.exp(-1j*phi/2)*np.cos(theta)]])
    U=np.array([[np.exp(1j*phi/2)*np.cos(theta) ,-1j* np.exp(1j*zeta)*np.sin(theta)],[-1j* np.exp(-1j*zeta)*np.sin(theta),np.exp(-1j*phi/2)*np.cos(theta)]])
    #Nucrypt entangled system
    uhu=np.matmul(U,np.matmul(H_rho,np.transpose(np.conj(U)))) 
    uvu=np.matmul(U,np.matmul(V_rho,np.transpose(np.conj(U))))
    ulu=np.matmul(U,np.matmul(L_rho,np.transpose(np.conj(U))))
    uru=np.matmul(U,np.matmul(R_rho,np.transpose(np.conj(U))))
    uau=np.matmul(U,np.matmul(A_rho,np.transpose(np.conj(U))))
    udu=np.matmul(U,np.matmul(D_rho,np.transpose(np.conj(U))))


    '''
    uhu=U*H_rho*U.dag() np.transpose(np.conj(U))
    uvu=U*V_rho*U.dag() np.transpose(np.conj(U))
    ulu=U*L_rho*U.dag() np.transpose(np.conj(U))
    uru=U*R_rho*U.dag() np.transpose(np.conj(U))
    uau=U*A_rho*U.dag() np.transpose(np.conj(U))
    udu=U*D_rho*U.dag() np.transpose(np.conj(U))
    '''
    #print(uhu)
    # Projector_noise_matrix              
    P_noise =  np.array([[np.kron(H_rho,uhu),np.kron(H_rho,uvu),np.kron(V_rho,uvu),np.kron(V_rho,uhu),np.kron(V_rho,uru),np.kron(V_rho,ulu)],
                  [np.kron(H_rho,ulu),np.kron(H_rho,uru),np.kron(H_rho,udu),np.kron(H_rho,uau),np.kron(V_rho,uau),np.kron(V_rho,udu)],
                  [np.kron(A_rho,udu),np.kron(A_rho,uau),np.kron(D_rho,uau),np.kron(D_rho,udu),np.kron(D_rho,uru),np.kron(D_rho,ulu)],
                  [np.kron(A_rho,ulu),np.kron(A_rho,uru),np.kron(A_rho,uhu),np.kron(A_rho,uvu),np.kron(D_rho,uvu),np.kron(D_rho,uhu)],
                  [np.kron(R_rho,uhu),np.kron(R_rho,uvu),np.kron(L_rho,uvu),np.kron(L_rho,uhu),np.kron(L_rho,uru),np.kron(L_rho,ulu)],
                  [np.kron(R_rho,ulu),np.kron(R_rho,uru),np.kron(R_rho,udu),np.kron(R_rho,uau),np.kron(L_rho,uau),np.kron(L_rho,udu)]])
    #M_noise = (np.zeros((6,6),dtype = 'complex_'))
    #print((rho * P_noise[1][0].full()).trace())
    for i in range(6):
      for j in range(6):
        input_M_noisy[ni+k*n_noisy][0][i][j] = np.trace(np.matmul(rho , P_noise[i][j]))
        #print(input_M_noisy[ni+k*n_noisy][0][i][j])

  #print(tensor(H_rho,uhu))
  #print(P_noise[0][0])
  '''
  s00= Qobj([[M[0][0] + M[0][1] + M[0][3] + M[0][2]]])
  s12= Qobj([[ M[2][3] - M[2][5] - M[3][1] + M[3][0]]])
  s21= Qobj([[ M[5][2] - M[5][3] - M[5][5] + M[5][4]]])
  s23= Qobj([[ M[4][0] - M[4][1] - M[4][3] + M[4][2]]])
  s32= Qobj([[ M[1][1] - M[1][0] - M[0][4] + M[0][5]]])
  s01= Qobj([[ M[2][3] - M[2][2] + M[2][0] - M[2][1]]])
  s03= Qobj([[ M[0][0] - M[0][1] + M[0][3] - M[0][2]]])
  s20= Qobj([[ M[5][1] - M[4][4] + M[5][0] - M[4][5]]])
  ##
  s11= Qobj([[ M[2][3] - M[2][2] - M[2][0] + M[2][1]]])
  s13= Qobj([[ M[3][5] - M[3][4] - M[3][2] + M[3][3]]])
  s22= Qobj([[ M[5][1] - M[5][0] - M[4][4] + M[4][5]]])
  s31= Qobj([[ M[1][2] - M[1][3] - M[1][5] + M[1][4]]])
  s33= Qobj([[ M[0][0] - M[0][1] - M[0][3] + M[0][2]]])
  s02= Qobj([[ M[5][1] + M[4][4] - M[5][0] - M[4][5]]])
  s10= Qobj([[ M[2][3] + M[2][2] - M[2][0] - M[2][1]]])
  s30= Qobj([[ M[0][0] + M[0][1] - M[0][3] - M[0][2]]])
  '''
  s00= M[0][0] + M[0][1] + M[0][3] + M[0][2]
  s12=  M[2][3] - M[2][5] - M[3][1] + M[3][0]
  s21=  M[5][2] - M[5][3] - M[5][5] + M[5][4]
  s23= M[4][0] - M[4][1] - M[4][3] + M[4][2]
  s32=  M[1][1] - M[1][0] - M[0][4] + M[0][5]
  s01= M[2][3] - M[2][2] + M[2][0] - M[2][1]
  s03=  M[0][0] - M[0][1] + M[0][3] - M[0][2]
  s20= M[5][1] - M[4][4] + M[5][0] - M[4][5]
  ##
  s11=  M[2][3] - M[2][2] - M[2][0] + M[2][1]
  s13=  M[3][5] - M[3][4] - M[3][2] + M[3][3]
  s22=  M[5][1] - M[5][0] - M[4][4] + M[4][5]
  s31=  M[1][2] - M[1][3] - M[1][5] + M[1][4]
  s33= M[0][0] - M[0][1] - M[0][3] + M[0][2]
  s02= M[5][1] + M[4][4] - M[5][0] - M[4][5]
  s10=  M[2][3] + M[2][2] - M[2][0] - M[2][1]
  s30=  M[0][0] + M[0][1] - M[0][3] - M[0][2]

  S= [s00,s01,s02,s03,s10,s11,s12,s13,s20,s21,s22,s23,s30,s31,s32,s33]

  #basis = pauli_basis(2)
  #rho_sqt= Qobj(np.zeros((4,4)))
  ##rho_sqt= np.zeros((4,4))
  #for i in range(16):
    #rho_sqt += (0.5*S[i]*basis[i])
  
  #print("RHO _ SQT :",rho_sqt.data)
  #print("RHO _ ORIGINAL :",rho.data)
  #print("Infidelity : ",fidelity(rho,rho_sqt))

  # Tau-matrix

  det_rho = np.linalg.det(rho)
  m1_00= np.linalg.det(matrix_minor(rho, 0, 0))
  m1_01= np.linalg.det(matrix_minor(rho, 0, 1))
  m2_0011= np.linalg.det(matrix_minor(matrix_minor(rho, 0, 0),0,0))
  m2_0112= np.linalg.det(matrix_minor(matrix_minor(rho, 0, 1),0,1))
  m2_0012= np.linalg.det(matrix_minor(matrix_minor(rho, 0, 0),0,1))
  rho_t=rho
  '''
  rho_t = rho.full()
  
  tau= Qobj([[np.sqrt(det_rho/m1_00),0,0,0],
            [m1_01/np.sqrt(m1_00*m2_0011),np.sqrt(m1_00/m2_0011),0,0], 
            [m2_0112/(np.sqrt(rho_t[3][3]*m2_0011)),m2_0012/(np.sqrt(rho_t[3][3]*m2_0011)),np.sqrt(m2_0011/rho_t[3][3]),0],
            [rho_t[3][0]/np.sqrt(rho_t[3][3]),rho_t[3][1]/np.sqrt(rho_t[3][3]),rho_t[3][2]/np.sqrt(rho_t[3][3]),np.sqrt(rho_t[3][3])]])
  
  tau_np=tau.full()
  '''
  tau= np.array([[np.sqrt(det_rho/m1_00),0,0,0],
          [m1_01/np.sqrt(m1_00*m2_0011),np.sqrt(m1_00/m2_0011),0,0], 
          [m2_0112/(np.sqrt(rho_t[3][3]*m2_0011)),m2_0012/(np.sqrt(rho_t[3][3]*m2_0011)),np.sqrt(m2_0011/rho_t[3][3]),0],
          [rho_t[3][0]/np.sqrt(rho_t[3][3]),rho_t[3][1]/np.sqrt(rho_t[3][3]),rho_t[3][2]/np.sqrt(rho_t[3][3]),np.sqrt(rho_t[3][3])]])
  tau_np=tau
  tau_0= tau_np[0][0].real;tau_1=tau_np[1][1].real;tau_2=tau_np[2][2].real;tau_3=tau_np[3][3].real;
  tau_4=tau_np[1][0].real;tau_5=tau_np[1][0].imag;tau_6=tau_np[2][1].real;tau_7=tau_np[2][1].imag;
  tau_8=tau_np[3][2].real;tau_9=tau_np[3][2].imag;tau_10=tau_np[2][0].real;tau_11=tau_np[2][0].imag;
  tau_12=tau_np[3][1].real;tau_13=tau_np[3][1].imag;tau_14=tau_np[3][0].real;tau_15=tau_np[3][0].imag 

  # Creating dataset Tau labels
  for kk in range(n_noisy):
    labels_Tau[kk+n_noisy*k][0] = tau_0;labels_Tau[kk+n_noisy*k][1] = tau_1;labels_Tau[kk+n_noisy*k][2] = tau_2;labels_Tau[kk+n_noisy*k][3] = tau_3;
    labels_Tau[kk+n_noisy*k][4] = tau_4;labels_Tau[kk+n_noisy*k][5] = tau_5;labels_Tau[kk+n_noisy*k][6] = tau_6;labels_Tau[kk+n_noisy*k][7] = tau_7;
    labels_Tau[kk+n_noisy*k][8] = tau_8;labels_Tau[kk+n_noisy*k][9] = tau_9;labels_Tau[kk+n_noisy*k][10] = tau_10;labels_Tau[kk+n_noisy*k][11] = tau_11;
    labels_Tau[kk+n_noisy*k][12] = tau_12;labels_Tau[kk+n_noisy*k][13] = tau_13;labels_Tau[kk+n_noisy*k][14] = tau_14;labels_Tau[kk+n_noisy*k][15] = tau_15
    #labels_Tau[kk+n_noisy*k][i][j] = tau_np[i][j]
    #print(labels_Tau[i][j][k])


  # Reconstructing rho
  #rho_tau = (tau.dag() * tau)/ (tau.dag() * tau).tr()
  rho_tau = (np.matmul(np.transpose(np.conj(tau)),tau))/ np.trace(np.matmul(np.transpose(np.conj(tau)),tau))
  #rho_tau = (tau.dag() * tau)
  print(fidelity(Qobj(rho),Qobj(rho_tau)))
  #print(rho_tau)
  #print(rho)
      


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:24: ComplexWarning: Casting complex values to real discards the imaginary part


1.000000000000003
1.0000000000000002
0.9999999999999976
0.9999999999999998
1.0000000000000013
0.9999999999999999
1.0000000000000004
1.0000000000000004
0.9999999999999998
0.9999999999999994
0.9999999999999993
1.0000000000000109
0.9999999999999992
1.0000000000000002
0.9999999999999994
1.000000000000004
1.0000000000000007
1.000000000000001
0.999999999999998
1.0000000000000004
1.0000000000000009
1.0000000000000002
1.0
0.9999999999999992
1.000000000000001
0.9999999999999979
0.9999999999999996
0.9999999999999996
1.0000000000000009
1.0
1.0
0.9999999999999998
0.9999999999999942
0.9999999999999991
1.0000000000000022
1.0
0.9999999999999992
0.9999999999999994
0.9999999999999978
0.9999999999999993
1.0000000000000004
0.9999999999999998
0.9999999999999998
0.9999999999999998
1.0000000000001106
1.0000000000000009
0.9999999999999988
0.9999999999999961
0.9999999999999989
1.00000000000001
0.999999999999998
0.9999999999999989
0.9999999999999998
0.9999999999999989
0.9999999999999994
1.0000000000000004
0.99

In [ ]:
#split data train : test
noisy_test=5
noisy_train=195
train_dataset_index=int(dataset * 0.975)
test_dataset_index=int(dataset)
test_dataset=int(dataset-train_dataset_index)
train_x_a=[]
test_x_a=[]
train_y_a=[]
test_y_a=[]
print(labels_Tau.shape)

for kkk in range(n_noisy):
  train_x_a.append(input_M_noisy[kkk*(n_noisy):kkk*(n_noisy)+noisy_train][:][:][:])
  test_x_a.append(input_M_noisy[kkk*(n_noisy)+noisy_train:kkk*(n_noisy)+n_noisy][:][:][:])
  train_y_a.append(labels_Tau[kkk*(n_noisy):kkk*(n_noisy)+noisy_train][:])
  test_y_a.append(labels_Tau[kkk*(n_noisy)+noisy_train:kkk*(n_noisy)+n_noisy][:])




train_x_a=np.array(train_x_a)
test_x_a=np.array(test_x_a)
train_y_a=np.array(train_y_a)
test_y_a=np.array(test_y_a)
print(test_y_a.shape)
train_x_a = train_x_a.reshape(-1,1,6,6)
test_x_a = test_x_a.reshape(-1,1,6,6)
train_y_a = train_y_a.reshape(-1,16)
test_y_a = test_y_a.reshape(-1,16)


#print(test_y_a[999][:][:][:]) 

train_x=torch.from_numpy(train_x_a.real).float()
train_y=torch.from_numpy(train_y_a).float()
test_x=torch.from_numpy(test_x_a.real).float()
test_y=torch.from_numpy(test_y_a).float()
print(train_x.shape)
print(train_y.shape)
print(test_x.shape)

#print(test_x[994][:][:][:])
#print(input_M_noisy[39799][:][:][:])


#print(test_y[994][:][:][:])
#print(labels_Tau[39799][:][:][:])




(40000, 16)
(200, 5, 16)
torch.Size([39000, 1, 6, 6])
torch.Size([39000, 16])
torch.Size([1000, 1, 6, 6])


In [ ]:
from torch.utils.data import DataLoader, Dataset
import torchvision.transforms as T
trainset= {'train': train_x, 'label': train_y }
testset= {'test': test_x, 'label': test_y }
class CustomTextDataset(Dataset):
    def __init__(self, txt, labels):
        self.labels = labels
        self.text = txt
    def __len__(self):
            return len(self.labels)
    def __getitem__(self, idx):
            label = self.labels[idx]
            text = self.text[idx]
            sample = {"X": text, "Y": label}
            return sample

transformed_dataset = CustomTextDataset(trainset['train'],trainset['label'])
train_dl = DataLoader(transformed_dataset, batch_size=4, shuffle=True, num_workers=2, pin_memory=True)


test_transformed_dataset = CustomTextDataset(testset['test'],testset['label'])
test_dl = DataLoader(test_transformed_dataset, batch_size=4, shuffle=True, num_workers=2, pin_memory=True)
# Display text and label.
print('\nFirst iteration of data set: ', next(iter(train_dl)), '\n')
# Print how many items are in the data set
print('Length of data set: ', len(train_dl), '\n')
print('\nFirst iteration of data set: ', next(iter(test_dl)), '\n')
# Print how many items are in the data set
print('Length of data set: ', len(test_dl), '\n')

print(shape(next(iter(train_dl))['X']))



First iteration of data set:  {'X': tensor([[[[0.3742, 0.3013, 0.1828, 0.1418, 0.1535, 0.1710],
          [0.0889, 0.5865, 0.2626, 0.4129, 0.3034, 0.0211],
          [0.1294, 0.5200, 0.1963, 0.1543, 0.2756, 0.0750],
          [0.1850, 0.4645, 0.3251, 0.3243, 0.1597, 0.1909],
          [0.3340, 0.2102, 0.2739, 0.1820, 0.3646, 0.0913],
          [0.1686, 0.3755, 0.1187, 0.4254, 0.2908, 0.1650]]],


        [[[0.2493, 0.2435, 0.1539, 0.3534, 0.2517, 0.2556],
          [0.3733, 0.1195, 0.3074, 0.1853, 0.2011, 0.3062],
          [0.3592, 0.2196, 0.1668, 0.2544, 0.2267, 0.1945],
          [0.4343, 0.1445, 0.3890, 0.1898, 0.2076, 0.2136],
          [0.1152, 0.1824, 0.2150, 0.4874, 0.2964, 0.4061],
          [0.2228, 0.0748, 0.1457, 0.1519, 0.2346, 0.4679]]],


        [[[0.2819, 0.2868, 0.2656, 0.1656, 0.3130, 0.1183],
          [0.2337, 0.3351, 0.4453, 0.1235, 0.1877, 0.2436],
          [0.2819, 0.1383, 0.1729, 0.4070, 0.4225, 0.1573],
          [0.1946, 0.2255, 0.1872, 0.2330, 0.3194, 0.26

In [ ]:
partition= {'train': train_x, 'validation': test_x }
for i in range(n_rhos*n_noisy):
  labels={'train_x':train_y , 'test_x':test_y}

print(partition)

{'train': tensor([[[[0.0805, 0.2574, 0.3567, 0.3054, 0.2329, 0.4292],
          [0.1367, 0.2012, 0.2039, 0.1340, 0.2117, 0.4504],
          [0.1351, 0.1272, 0.2185, 0.5192, 0.3133, 0.4244],
          [0.1415, 0.1208, 0.0921, 0.1702, 0.4440, 0.2937],
          [0.2402, 0.4573, 0.1568, 0.1456, 0.1220, 0.1805],
          [0.3855, 0.3121, 0.4114, 0.2862, 0.0595, 0.2430]]],


        [[[0.0950, 0.2429, 0.3430, 0.3191, 0.3769, 0.2852],
          [0.1011, 0.2367, 0.1642, 0.1737, 0.1817, 0.4804],
          [0.1426, 0.1197, 0.2357, 0.5020, 0.4754, 0.2623],
          [0.1240, 0.1383, 0.0929, 0.1694, 0.4165, 0.3212],
          [0.2524, 0.4452, 0.1407, 0.1617, 0.2088, 0.0937],
          [0.2927, 0.4049, 0.4166, 0.2809, 0.0745, 0.2279]]],


        [[[0.0880, 0.2499, 0.3141, 0.3480, 0.3437, 0.3184],
          [0.2177, 0.1201, 0.2029, 0.1350, 0.1758, 0.4863],
          [0.1512, 0.1111, 0.1997, 0.5380, 0.3660, 0.3717],
          [0.1644, 0.0979, 0.1193, 0.1430, 0.4210, 0.3167],
          [0.3014, 0.3

In [ ]:
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 25, 3 ,stride=1,padding='same')
        #self.pool = nn.MaxPool2d(2, stride=2)
        #self.zp=nn.ZeroPad2d(1)
        self.conv2 = nn.Conv2d(25, 80, 3, stride=1,padding='same')
        
        self.fc1 = nn.Linear(6*6*80, 450)
        self.do = nn.Dropout(0.25)
        self.fc2 = nn.Linear(450, 16)
        #self.fc3 = nn.Linear(450, 16)

    def forward(self, x):
        x = (F.relu(self.conv1(x)))
        #x= self.zp(x)
        #print(x.shape)
        x = F.relu(self.conv2(x))
        #x= self.zp(x)
        #print(x.shape)
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = self.do(x)
        x = F.relu(self.fc1(x))
        x = self.do(x)
        #x = F.relu(self.fc2(x))
        x = self.fc2(x)
        #x = self.fc3(x)

        return x


net = Net()

In [ ]:
import torch.optim as optim

criterion = nn.MSELoss()
optimizer = optim.SGD(net.parameters(), lr=0.05)

In [ ]:
'''
checkpoint = torch.load(PATH)
net.load_state_dict(checkpoint['state_dict'])
optimizer.load_state_dict(checkpoint['optimizer'])
epoch = checkpoint['epoch']
'''

"\ncheckpoint = torch.load(PATH)\nnet.load_state_dict(checkpoint['state_dict'])\noptimizer.load_state_dict(checkpoint['optimizer'])\nepoch = checkpoint['epoch']\n"

In [ ]:
tau_regen=np.zeros((train_dataset_index,16))
batch_s=int(4)
tau_regen_batch=np.zeros((batch_s,16))
rho_tau_regen_batch=np.zeros((batch_s,16))
batch_elements= int(train_dataset_index/batch_s)
n_epochs=10
#train_x_batch = train_x[0:batch_s][:][:][:]
#train_y_batch = train_y[0:batch_s][:][:]


for epoch in range(n_epochs):  # loop over the dataset multiple times

    running_loss = 0.0
    #for j in range(batch_elements):
    for j,train_x_batch in enumerate(train_dl):

        #train_x_batch = train_x[j*batch_s:(j+1)*batch_s][:][:][:]
        #train_y_batch = train_y[j*batch_s:(j+1)*batch_s][:]
        # get the inputs; data is a list of [inputs, labels]

        inputs, labels = train_x_batch['X'],train_x_batch['Y']
        #print(labels.shape)
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        net.train()
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        #print(loss.item())
  
        if j % batch_elements == batch_elements-1:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, j + 1, running_loss / batch_elements))
            running_loss = 0.0

print('Finished Training')


[1,  9750] loss: 0.004
[2,  9750] loss: 0.004
[3,  9750] loss: 0.004
[4,  9750] loss: 0.004
[5,  9750] loss: 0.004
[6,  9750] loss: 0.004
[7,  9750] loss: 0.004
[8,  9750] loss: 0.003
[9,  9750] loss: 0.003
[10,  9750] loss: 0.003
Finished Training


In [ ]:
PATH = './MLAQSE.pth'
checkpoint = {
    'epoch': epoch + 1,
    'state_dict': net.state_dict(),
    'optimizer': optimizer.state_dict()}
torch.save(checkpoint, PATH)

In [ ]:
net.eval()
outputs = net(test_x)



In [ ]:
print(outputs[2])
print(test_y[2])
print(train_y[2])

tensor([ 1.1823e-01,  3.4803e-01,  5.2579e-01,  5.2542e-01,  4.1782e-02,
         2.9488e-02,  3.7500e-04, -1.5222e-01,  2.6151e-01, -1.1513e-01,
         8.9580e-02,  9.3469e-02,  1.9560e-01,  2.2463e-01,  2.1824e-01,
        -2.7630e-02], grad_fn=<SelectBackward>)
tensor([ 0.0932,  0.3495,  0.5531,  0.5273,  0.0368,  0.0315,  0.0087, -0.1469,
         0.2570, -0.1101,  0.1061,  0.0945,  0.2199,  0.2461,  0.2292, -0.0377])
tensor([ 0.0932,  0.3495,  0.5531,  0.5273,  0.0368,  0.0315,  0.0087, -0.1469,
         0.2570, -0.1101,  0.1061,  0.0945,  0.2199,  0.2461,  0.2292, -0.0377])


In [ ]:

outputs = net(train_x)

#print(test_y.shape)
train_tau_regen=np.zeros((train_dataset_index,4,4),dtype="complex_")
#tau_pred=np.array((5000,4,4))
rho_pred=[]

for i in range(train_dataset_index):
  train_tau_regen[i][0][0]= outputs[i][0]
  train_tau_regen[i][1][0]= outputs[i][4]+ 1j*outputs[i][5]
  train_tau_regen[i][2][0]= outputs[i][10]+ 1j*outputs[i][11]
  train_tau_regen[i][3][0]= outputs[i][14]+ 1j*outputs[i][15]
  train_tau_regen[i][1][1]= outputs[i][1]
  train_tau_regen[i][2][1]= outputs[i][6]+ 1j*outputs[i][7]
  train_tau_regen[i][3][1]= outputs[i][12]+ 1j*outputs[i][13]
  train_tau_regen[i][2][2]= outputs[i][2]
  train_tau_regen[i][3][2]= outputs[i][8]+ 1j*outputs[i][9]
  train_tau_regen[i][3][3]= outputs[i][3]
  #tau_pred=Qobj(test_tau_regen[i])
  #rho_pred.append( (tau_pred.dag()*tau_pred)/((tau_pred.dag()*tau_pred).tr()) )
  tau_pred=train_tau_regen[i]
  rho_pred.append((np.matmul(np.transpose(np.conj(tau_pred)),tau_pred))/np.trace(np.matmul(np.transpose(np.conj(tau_pred)),tau_pred)) )

#print(rho_pred)

train_tau_groundtruth=np.zeros((train_dataset_index,4,4),dtype="complex_")
rho_target=[]
for i in range(train_dataset_index):
  train_tau_groundtruth[i][0][0]= train_y_a[i][0]
  train_tau_groundtruth[i][1][0]= train_y_a[i][4]+ 1j*train_y_a[i][5]
  train_tau_groundtruth[i][2][0]= train_y_a[i][10]+ 1j*train_y_a[i][11]
  train_tau_groundtruth[i][3][0]= train_y_a[i][14]+ 1j*train_y_a[i][15]
  train_tau_groundtruth[i][1][1]= train_y_a[i][1]
  train_tau_groundtruth[i][2][1]= train_y_a[i][6]+ 1j*train_y_a[i][7]
  train_tau_groundtruth[i][3][1]= train_y_a[i][12]+ 1j*train_y_a[i][13]
  train_tau_groundtruth[i][2][2]= train_y_a[i][2]
  train_tau_groundtruth[i][3][2]= train_y_a[i][8]+ 1j*train_y_a[i][9]
  train_tau_groundtruth[i][3][3]= train_y_a[i][3]
  #tau_target=Qobj(test_tau_groundtruth[i])
  #rho_target.append( (tau_target.dag()*tau_target)/((tau_target.dag()*tau_target).tr()) )
  tau_target=train_tau_groundtruth[i]
  rho_target.append(np.matmul(np.transpose(np.conj(tau_target)),tau_target)/np.trace(np.matmul(np.transpose(np.conj(tau_target)),tau_target)) )

print(rho_target[0])
print(rho_pred[0])


running_fidelity=0
for i in range(train_dataset_index):
  running_fidelity+= fidelity(Qobj(rho_pred[i]),Qobj(rho_target[i]))
Avg_fid=running_fidelity/train_dataset_index
print(Avg_fid)


[[0.08517784+0.j         0.04104538+0.03725158j 0.12174869-0.06782409j
  0.12082952+0.01986334j]
 [0.04104538-0.03725158j 0.25270413+0.j         0.03426871-0.00621307j
  0.11594078-0.12973982j]
 [0.12174869+0.06782409j 0.03426871+0.00621307j 0.38411493+0.j
  0.13551902+0.05802526j]
 [0.12082952-0.01986334j 0.11594078+0.12973982j 0.13551902-0.05802526j
  0.2780031 +0.j        ]]
[[0.08820633+0.j         0.04732665+0.02938806j 0.11545305-0.08062174j
  0.12422085+0.00911894j]
 [0.04732665-0.02938806j 0.24766917+0.j         0.04413595-0.01020654j
  0.11783468-0.1109796j ]
 [0.11545305+0.08062174j 0.04413595+0.01020654j 0.38071951+0.j
  0.15056868+0.06416709j]
 [0.12422085-0.00911894j 0.11783468+0.1109796j  0.15056868-0.06416709j
  0.283405  +0.j        ]]
0.9922876602335116


In [ ]:
outputs = net(test_x)

#print(test_y.shape)
test_tau_regen=np.zeros((test_dataset,4,4),dtype="complex_")
#tau_pred=np.array((5000,4,4))
rho_pred=[]

for i in range(test_dataset):
  test_tau_regen[i][0][0]= outputs[i][0]
  test_tau_regen[i][1][0]= outputs[i][4]+ 1j*outputs[i][5]
  test_tau_regen[i][2][0]= outputs[i][10]+ 1j*outputs[i][11]
  test_tau_regen[i][3][0]= outputs[i][14]+ 1j*outputs[i][15]
  test_tau_regen[i][1][1]= outputs[i][1]
  test_tau_regen[i][2][1]= outputs[i][6]+ 1j*outputs[i][7]
  test_tau_regen[i][3][1]= outputs[i][12]+ 1j*outputs[i][13]
  test_tau_regen[i][2][2]= outputs[i][2]
  test_tau_regen[i][3][2]= outputs[i][8]+ 1j*outputs[i][9]
  test_tau_regen[i][3][3]= outputs[i][3]
  #tau_pred=Qobj(test_tau_regen[i])
  #rho_pred.append( (tau_pred.dag()*tau_pred)/((tau_pred.dag()*tau_pred).tr()) )
  tau_pred=test_tau_regen[i]
  rho_pred.append((np.matmul(np.transpose(np.conj(tau_pred)),tau_pred))/np.trace(np.matmul(np.transpose(np.conj(tau_pred)),tau_pred)) )

#print(rho_pred)


In [ ]:

test_tau_groundtruth=np.zeros((test_dataset,4,4),dtype="complex_")
rho_target=[]
for i in range(test_dataset):
  test_tau_groundtruth[i][0][0]= test_y_a[i][0]
  test_tau_groundtruth[i][1][0]= test_y_a[i][4]+ 1j*test_y_a[i][5]
  test_tau_groundtruth[i][2][0]= test_y_a[i][10]+ 1j*test_y_a[i][11]
  test_tau_groundtruth[i][3][0]= test_y_a[i][14]+ 1j*test_y_a[i][15]
  test_tau_groundtruth[i][1][1]= test_y_a[i][1]
  test_tau_groundtruth[i][2][1]= test_y_a[i][6]+ 1j*test_y_a[i][7]
  test_tau_groundtruth[i][3][1]= test_y_a[i][12]+ 1j*test_y_a[i][13]
  test_tau_groundtruth[i][2][2]= test_y_a[i][2]
  test_tau_groundtruth[i][3][2]= test_y_a[i][8]+ 1j*test_y_a[i][9]
  test_tau_groundtruth[i][3][3]= test_y_a[i][3]
  #tau_target=Qobj(test_tau_groundtruth[i])
  #rho_target.append( (tau_target.dag()*tau_target)/((tau_target.dag()*tau_target).tr()) )
  tau_target=test_tau_groundtruth[i]
  rho_target.append(np.matmul(np.transpose(np.conj(tau_target)),tau_target)/np.trace(np.matmul(np.transpose(np.conj(tau_target)),tau_target)) )

print(rho_target[0])
print(rho_pred[0])


[[0.08517784+0.j         0.04104538+0.03725158j 0.12174869-0.06782409j
  0.12082952+0.01986334j]
 [0.04104538-0.03725158j 0.25270413+0.j         0.03426871-0.00621307j
  0.11594078-0.12973982j]
 [0.12174869+0.06782409j 0.03426871+0.00621307j 0.38411493+0.j
  0.13551902+0.05802526j]
 [0.12082952-0.01986334j 0.11594078+0.12973982j 0.13551902-0.05802526j
  0.2780031 +0.j        ]]
[[0.09210051+0.j         0.03406116+0.01762217j 0.10962434-0.07410885j
  0.10425217+0.01800505j]
 [0.03406116-0.01762217j 0.22793575+0.j         0.03796872-0.01006027j
  0.12576057-0.11152444j]
 [0.10962434+0.07410885j 0.03796872+0.01006027j 0.37137264+0.j
  0.14222735+0.04506199j]
 [0.10425217-0.01800505j 0.12576057+0.11152444j 0.14222735-0.04506199j
  0.3085911 +0.j        ]]


In [ ]:
running_fidelity=0
for i in range(test_dataset):
  running_fidelity+= fidelity(Qobj(rho_pred[i]),Qobj(rho_target[i]))
Avg_fid=running_fidelity/test_dataset
print(Avg_fid)

0.9899485977552153
